In [1]:
import numpy as np
from scipy.stats import pearsonr
import csv, random

In [2]:
num_user = 0
num_movie = 0
similarity_list = []
mean_list = []

rating_data = None
movie_data = None
movie_link_data = None

movie_id_index_dic = {}
chat_id_user_id_dic = {}

#read three csv file
with open("C:/Users/kingy/Desktop/ml-latest-small/ratings.csv") as f:
    reader = csv.reader(f)
    rating_data = [row for row in  reader]
    num_user = int(rating_data[len(rating_data) - 1][0])

with open("C:/Users/kingy/Desktop/ml-latest-small/movies.csv", encoding='utf-8') as f:
    reader = csv.reader(f)
    movie_data = [row for row in  reader]
    num_movie = len(movie_data) - 1

with open("C:/Users/kingy/Desktop/ml-latest-small/links.csv", encoding='utf-8') as f:
    reader = csv.reader(f)
    movie_link_data = [row for row in  reader]
    
# get the dict for movie id => index
for i in range(len(movie_data)):
    if i == 0:
        continue
    else:    
        movie_id_index_dic[movie_data[i][0]] = i
        
# get the dict for movie id => link
for i in range(len(movie_data)):
    if i == 0:
        continue
    else:    
        movie_id_index_dic[movie_data[i][0]] = i

In [3]:
matrix = np.zeros((num_user,num_movie), dtype=int)

In [4]:
for i in range(len(rating_data)):
    if i == 0:
        continue
    else:
        matrix[int(rating_data[i][0]) - 1][int(movie_id_index_dic[(rating_data[i][1])]) - 1] = int(float(rating_data[i][2]))

In [5]:
# similarity between users
def similarity(u1, u2):
    r, _ = pearsonr(matrix[u1], matrix[u2])
    return r

In [6]:
# get one unrated movie (random)
def get_unrated_movie(user_id):
    user_list = matrix[user_id - 1]
    unrated_list = []
    for i in range(len(user_list)):
        if user_list[i] == 0:
            unrated_list.append(i)
        else:
            continue
    unrated_random = random.randint(0,len(unrated_list) - 1)
    movie_index = unrated_list[unrated_random]
    
    movie_id = movie_data[1 + movie_index][0]
    movie_title = movie_data[1 + movie_index][1]
    movie_url = "https://www.imdb.com/title/tt0" + str(movie_link_data[1 + movie_index][1]) + "/"
    return "{\"title\":\"" + movie_title + "\",\"url\":\"" + movie_url + "\",\"movie_id\":\"" + movie_id + "\"}"

# get all unrated movie (for recommend)
def get_all_unrated_movie(user_id):
    user_list = matrix[user_id - 1]
    unrated_list = []
    for i in range(len(user_list)):
        if user_list[i] == 0:
            unrated_list.append(i)
        else:
            continue
    return unrated_list

In [7]:
def rate_movie(user_id, movie_index, rating):
    matrix[user_id - 1, movie_index - 1] = rating

In [8]:
def register(chat_id):
    global matrix, num_user, chat_id_user_id_dic
    if chat_id_user_id_dic.__contains__(chat_id) == False:
        # add a new row on the matrix, then increase the num_user and put the new user into the dict
        matrix = np.row_stack((matrix, np.zeros((1, num_movie), dtype=int)))
        num_user+=1
        chat_id_user_id_dic[chat_id] = num_user
        return "{\"exists\": 0}"
    else:
        return "{\"exists\": 1}"

In [9]:
# predict a user's rating on one movie
def predict(user_id, movie_index):
    global similarity_list, mean_list
    #real index in the python list, start with 0
    movie_index-=1
    user_id-=1

    x = 0
    y = 0
    for i in range(num_user):
        if i != (user_id):
            x+=(similarity_list[i]*(matrix[i][movie_index] - mean_list[i]))
            y+= abs(similarity_list[i])

    pred_rating = np.mean([r for r in matrix[user_id] if r > 0]) + x/y
    return pred_rating


def cal_similarity(user_id):
    global similarity_list
    user_id-=1
    
    for i in range(num_user):
        if i != (user_id):
            similarity_list.append(similarity(user_id, i))
        else:
            similarity_list.append(1)

def cal_mean(user_id):
    global mean_list
    user_id-=1
    
    for i in range(num_user):
        if i != (user_id):
            mean_list.append(np.mean([r for r in matrix[i] if r > 0]))
        else:
            mean_list.append(0)

In [10]:
# from large to small
def bubbleSort(list1, list2):
    for i in range(len(list1)-1):
        for j in range(len(list1)-i-1):
            if list1[j] < list1[j+1]:
                list1[j + 1], list1[j] = list1[j], list1[j + 1]
                list2[j + 1], list2[j] = list2[j], list2[j + 1]
    return list1,list2

In [ ]:
import json
from flask import Flask, request, jsonify

app = Flask(__name__)


@app.route('/register',methods=["POST"])
def f_register():
    data = request.get_data()
    json_data = json.loads(data)
    chat_id = json_data.get("chat_id")
    return register(int(chat_id))

@app.route('/get_unrated_movie',methods=["POST"])
def f_get_unrated_movie():
    data = request.get_data()
    json_data = json.loads(data)
    chat_id = json_data.get("chat_id")
    if chat_id_user_id_dic.__contains__(int(chat_id)) == False:
        return "{}"
    else:
        return get_unrated_movie(chat_id_user_id_dic[int(chat_id)])

@app.route('/rate_movie',methods=["POST"])
def f_rate_movie():
    data = request.get_data()
    json_data = json.loads(data)
    chat_id = json_data.get("chat_id")
    movie_id = json_data.get("movie_id")
    rating = json_data.get("rating")
    rate_movie(chat_id_user_id_dic[chat_id], movie_id_index_dic[movie_id], rating)
    return ""

@app.route('/recommend',methods=["POST"])
def f_recommend():
    global movie_data, movie_link_data

    data = request.get_data()
    json_data = json.loads(data)
    chat_id = json_data.get("chat_id")
    top_n = json_data.get("top_n")
    
    if chat_id_user_id_dic.__contains__(int(chat_id)) == False:
        return "{}"
    else:
        user_id = chat_id_user_id_dic[int(chat_id)]
        unseen_movie_list = get_all_unrated_movie(user_id)
        recommend_movie_list = []
        if (num_movie - len(unseen_movie_list)) < 10:
            ;
        else:
            pred_list = []
            cal_similarity(user_id)
            cal_mean(user_id)
            for movie_index in unseen_movie_list:
                pred_list.append(predict(user_id, movie_index))
            bubbleSort(pred_list, unseen_movie_list)
            unseen_movie_index_top_list = unseen_movie_list[0:(top_n)]

            for i in range(top_n):
                recommend_movie_list.append(
                    {"title":movie_data[1 + unseen_movie_index_top_list[i]][1],
                     "url":"https://www.imdb.com/title/tt0" + str(movie_link_data[1 + unseen_movie_index_top_list[i]][1]) + "/"
                    })
        return json.dumps({"movies": recommend_movie_list})

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Nov/2018 21:31:39] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:31:51] "POST /get_unrated_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:31:53] "POST /rate_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:31:56] "POST /get_unrated_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:01] "POST /get_unrated_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:03] "POST /rate_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:05] "POST /get_unrated_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:09] "POST /rate_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:13] "POST /get_unrated_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:15] "POST /rate_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:17] "POST /get_unrated_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:20] "POST /rate_movie HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2018 21:32:22] "POST /get_u

In [1]:
def predict_old(user_id, movie_index):
    #real index in the python list, start with 0
    movie_index-=1
    user_id-=1

    x = 0
    y = 0
    for i in range(num_user):
        if i != (user_id):
            x+=(similarity(user_id, i)*(matrix[i][movie_index] - np.mean([r for r in matrix[i] if r > 0])))
            y+= (similarity(user_id, i))

    pred_rating = np.mean([r for r in matrix[user_id] if r > 0]) + x/y
    return pred_rating
